In [1]:
import pandas as pd
import re
import nltk
from tqdm import tqdm
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np
from datasets import load_dataset

model_path = "/kaggle/input/distillbert_trained_on_marco/transformers/default/1/trained_biencoder_distilbert_better/encoder"
tokenizer_path = "/kaggle/input/distillbert_trained_on_marco/transformers/default/1/trained_biencoder_distilbert_better/tokenizer"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
max_length = 256
top_k = 5


model = AutoModel.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model.eval()


def embed_texts(texts, batch_size=64):
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt").to(device)
            outputs = model(**inputs).last_hidden_state.mean(dim=1)
            embeddings.append(outputs.cpu())
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings


print("Loading Natural Questions...")
nq = load_dataset("nq_open", split="validation")  


queries = nq['question']
answers = nq['answer']
passages = []
for ans_list in answers:
    if isinstance(ans_list, list) and len(ans_list) > 0:
        passages.append(ans_list[0]) 
    else:
        passages.append("") 

print(f"Loaded {len(queries)} queries and passages.")
print("Embedding passages...")
passage_embeddings = embed_texts(passages)
print("Building retrieval index...")
index = NearestNeighbors(n_neighbors=top_k, metric="cosine")
index.fit(passage_embeddings)
print("Embedding queries...")
query_embeddings = embed_texts(queries)
print("Retrieving top-k...")
distances, indices = index.kneighbors(query_embeddings, return_distance=True)
recall_at_1 = 0
recall_at_5 = 0
total = len(queries)

for query_idx in range(total):
    retrieved_indices = indices[query_idx]
    correct_idx = query_idx  # because passage[i] corresponds to query[i] directly here
    
    if correct_idx == retrieved_indices[0]:
        recall_at_1 += 1
    if correct_idx in retrieved_indices:
        recall_at_5 += 1

recall_at_1 /= total
recall_at_5 /= total

print(f"\n--- Retrieval Results on Natural Questions ---")
print(f"Recall@1: {recall_at_1:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")

# Show 5 examples
print("\nShowing first 5 queries and retrievals:\n")
for i in range(5):
    print(f"Query {i+1}: {queries[i]}\n")
    for rank, idx in enumerate(indices[i]):
        print(f"Top {rank+1} passage: {passages[idx][:300]}...")  # Show first 300 chars
    print("-" * 80)


2025-04-26 21:13:28.896320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745702009.047274      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745702009.091457      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading Natural Questions...


README.md:   0%|          | 0.00/8.77k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/214k [00:00<?, ?B/s]

...

Loaded 3610 queries and passages.
Embedding passages...


Embedding: 100%|██████████| 57/57 [00:24<00:00,  2.28it/s]


Building retrieval index...
Embedding queries...


Embedding: 100%|██████████| 57/57 [00:26<00:00,  2.13it/s]


Retrieving top-k...

--- Retrieval Results on Natural Questions ---
Recall@1: 0.0693
Recall@5: 0.1452

Showing first 5 queries and retrievals:

Query 1: when was the last time anyone was on the moon

Top 1 passage: During the last Ice Age...
Top 2 passage: last book...
Top 3 passage: the following day...
Top 4 passage: in the winter...
Top 5 passage: the year of the Dog...
--------------------------------------------------------------------------------
Query 2: who wrote he ain't heavy he's my brother lyrics

Top 1 passage: I Write Sins Not Tragedies...
Top 2 passage: his brother...
Top 3 passage: Rich Mullins...
Top 4 passage: her husband Albert Brown...
Top 5 passage: A'ja Wilson...
--------------------------------------------------------------------------------
Query 3: how many seasons of the bastard executioner are there

Top 1 passage: "Killer Within"...
Top 2 passage: Later in the sixth season...
Top 3 passage: season two...
Top 4 passage: the Four Seasons...
Top 5 passage: in s

In [24]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np

model_path = "/kaggle/input/distillbert_trained_on_marco/transformers/default/1/trained_biencoder_distilbert_better/encoder"
tokenizer_path = "/kaggle/input/distillbert_trained_on_marco/transformers/default/1/trained_biencoder_distilbert_better/tokenizer"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
max_length = 256
top_k = 5

model = AutoModel.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model.eval()

def embed_texts(texts, batch_size=64):
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt").to(device)
            outputs = model(**inputs).last_hidden_state.mean(dim=1)
            embeddings.append(outputs.cpu())
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings

print("Loading HotpotQA validation set...")
hotpot_val = load_dataset("hotpot_qa", "fullwiki", split="validation")

queries = []
gold_titles = [] 
passage_texts = []
passage_titles = []

for example in tqdm(hotpot_val, desc="Processing Validation Data"):
    query = example['question']
    queries.append(query)

    support_titles = example['supporting_facts']['title']
    gold_titles.append(set(support_titles))

    for title, sentences_list in zip(example['context']['title'], example['context']['sentences']):
        paragraph = " ".join(sentences_list)
        paragraph = paragraph.strip().replace("\n", " ")
        if len(paragraph.split()) >= 5:
            passage_titles.append(title)
            passage_texts.append(paragraph)


print(f"Loaded {len(queries)} queries and {len(passage_texts)} passages.")
print("Embedding passages...")
passage_embeddings = embed_texts(passage_texts)
print("Building retrieval index...")
index = NearestNeighbors(n_neighbors=top_k, metric="cosine")
index.fit(passage_embeddings)
print("Embedding queries...")
query_embeddings = embed_texts(queries)
print("Retrieving top-k passages...")
distances, indices = index.kneighbors(query_embeddings, return_distance=True)

recall_at_1 = 0
recall_at_5 = 0
total = len(queries)

for query_idx in range(total):
    retrieved_indices = indices[query_idx]
    gold_set = gold_titles[query_idx]
    retrieved_title_list = [passage_titles[i] for i in retrieved_indices]

    if retrieved_title_list[0] in gold_set:
        recall_at_1 += 1
    if any(title in gold_set for title in retrieved_title_list):
        recall_at_5 += 1

recall_at_1 /= total
recall_at_5 /= total

print(f"\n--- HotpotQA Retrieval Results ---")
print(f"Recall@1: {recall_at_1:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")

# Show examples
print("\nShowing first 5 queries and their retrievals:\n")
for i in range(5):
    print(f"Query {i+1}: {queries[i]}")
    print(f"Gold Titles: {list(gold_titles[i])}")
    print("Top retrieved titles:")
    for rank, idx in enumerate(indices[i]):
        print(f"Top {rank+1}: {passage_titles[idx]}")
    print("-" * 80)

Loading HotpotQA validation set...


Processing Validation Data: 100%|██████████| 7405/7405 [00:01<00:00, 4011.32it/s]


Loaded 7405 queries and 73642 passages.
Embedding passages...


Embedding: 100%|██████████| 1151/1151 [11:06<00:00,  1.73it/s]


Building retrieval index...
Embedding queries...


Embedding: 100%|██████████| 116/116 [01:06<00:00,  1.75it/s]


Retrieving top-k passages...

--- HotpotQA Retrieval Results ---
Recall@1: 0.5263
Recall@5: 0.6705

Showing first 5 queries and their retrievals:

Query 1: Were Scott Derrickson and Ed Wood of the same nationality?
Gold Titles: ['Scott Derrickson', 'Ed Wood']
Top retrieved titles:
Top 1: Jan D'Alquen and Ron Eveslage
Top 2: Craig Marshall
Top 3: Greg Bautzer
Top 4: Greg Bautzer
Top 5: James Kerwin
--------------------------------------------------------------------------------
Query 2: What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Gold Titles: ['Kiss and Tell (1945 film)', 'Shirley Temple']
Top retrieved titles:
Top 1: Meet Corliss Archer (TV series)
Top 2: Elizabeth Norment
Top 3: Harper Marshall
Top 4: Sarah Coburn
Top 5: Lisa Howard (American actress)
--------------------------------------------------------------------------------
Query 3: What science fantasy young adult series, told in first person, has a set of companion bo